In [1]:
import sys
import math
import json
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno
%matplotlib inline
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)


In [2]:
df = pd.read_csv("data_before_normazised.csv")

In [3]:
df.head()

,Unnamed: 0,city,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,...,42200,42220,43760,44700,45000,46020,46380,46700,47300,49700
0,0,1124,180000.0,125800.00000,102.564103,90.752462,19,10.000000,6.000000,16.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,426,477500.0,600000.00000,528.221778,524.475525,2,1.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
2,2,360,450000.0,694801.66751,300.000000,428.895978,1,10.026163,12.991399,26.321098,...,0,0,0,0,0,0,0,0,0,0
3,3,1306,295000.0,250000.00000,185.897436,175.606641,3,10.026163,1.000000,3.000000,...,0,0,0,0,0,0,0,0,0,0
4,4,487,895000.0,579900.00000,382.910117,285.946746,3,3.000000,1.000000,1.000000,...,0,0,0,0,0,0,0,1,0,0


In [4]:
df.columns

Index(['Unnamed: 0', 'city', 'median_sale_price', 'median_list_price',
       'median_ppsf', 'median_list_ppsf', 'homes_sold', 'pending_sales',
       'new_listings', 'inventory', 'months_of_supply', 'median_dom',
       'avg_sale_to_list', 'sold_above_list', 'price_drops',
       'off_market_in_two_weeks', 'year', 'month', 'MORTGAGE30US',
       'PropertyType_All_Residential', 'PropertyType_Condo/Co-op',
       'PropertyType_Multi-Family(2-4 Unit)',
       'PropertyType_Single_Family_Residential', 'PropertyType_Townhouse ',
       'CA', 'California', 'Anaheim', 'Bakersfield', 'Chico', 'Clearlake',
       'El Centro', 'Eureka', 'Fresno', 'Hanford', 'Los Angeles', 'Madera',
       'Merced', 'Modesto', 'Napa', 'Oakland', 'Oxnard', 'Red Bluff',
       'Redding', 'Riverside', 'Sacramento', 'Salinas', 'San Diego',
       'San Francisco', 'San Jose', 'San Luis Obispo', 'San Rafael',
       'Santa Cruz', 'Santa Maria', 'Santa Rosa', 'Sonora', 'Stockton',
       'Susanville', 'Truckee', 'Ukiah

In [5]:
columns_of_intrest = ['city', 'state', 'state_code', 'property_type', 'median_sale_price', 'median_list_price', 'median_ppsf',
                      'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'months_of_supply',
                      'median_dom', 'avg_sale_to_list', 'sold_above_list', 'price_drops', 'off_market_in_two_weeks', 
                      'parent_metro_region', 'parent_metro_region_metro_code', 'year', 'month', 'MORTGAGE30US']
df = df[columns_of_intrest]
df


KeyError: "['state', 'state_code', 'property_type', 'parent_metro_region', 'parent_metro_region_metro_code'] not in index"

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.isnull().sum()/len(df)*100

In [ ]:
missing_vals = df.isna().sum()/len(df)*100
missing_vals = missing_vals[missing_vals > 0]
print('Percentage of Missing Values:')
print(missing_vals)
print()
print('median_list_price')
print(df['median_list_price'].value_counts())

The maximum number of house that were from city burbank,Mountain View, Spring Valley, El Cerrito and Live Oak(top5)

# Property_type feature cleaning

In [ ]:
df['property_type'].value_counts()

In [ ]:
df['property_type'].unique()

# This feature is Categorical featire with 5 unique labels, hence we need to change this to numeric feature, for this we are using one hot encoding.

In [ ]:
# performing one hot encoding 
df['property_type']=df['property_type'].replace(['All Residential'],"PropertyType_All_Residential")
df['property_type']=df['property_type'].replace(['Single Family Residential'],"PropertyType_Single_Family_Residential")
df['property_type']=df['property_type'].replace(['Townhouse'],"PropertyType_Condo/Co-op")
df['property_type']=df['property_type'].replace(['Multi-Family (2-4 Unit)'],"PropertyType_Multi-Family(2-4 Unit)")
df['property_type']=df['property_type'].replace(['Condo/Co-op'],"PropertyType_Townhouse ")
one_hot_property_type = pd.get_dummies(df['property_type'])
df = df.drop('property_type',axis = 1)
df = df.join(one_hot_property_type)

In [ ]:
df.head()

# Median_Sale_Price feature cleaning

In [ ]:
df['median_sale_price'].value_counts()

In [ ]:
df['median_sale_price']

In [ ]:
# checking for negative value in feature "median_sale_price"
(df['median_sale_price'] < 0).sum().sum()
    

In [ ]:
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_sale_price
df['median_sale_price'].plot(kind="box")

In [ ]:

count = 1;
min_value = sys.maxsize
max_value = -1
print(min_value)
for key, value in df['median_sale_price'].iteritems():
    if value < min_value:
      min_value = value

    if value > max_value:
      max_value = value
    
print(min_value)
print(max_value)

In [ ]:
#checking for how many value outliers dropped
median_sale_price_df=df['median_sale_price']
value=1389750.0
#maximum value after removing outliers
count=0
for i, number in df['median_sale_price'].iteritems():
    if number > value:
        count=count+1
print(count)
#Percentage of outliers dropped
print("Percentage of outliers dropped",str((count*100)/df['median_sale_price'].shape[0])) 

# median_list_price feature cleaning

In [ ]:
df['median_list_price'].value_counts()

In [ ]:
df['median_list_price'].unique()

In [ ]:
#checking for missing value 
missing_vals2 = df['median_list_price'].isna().sum()/len(df['median_sale_price'])*100
missing_vals2

In [ ]:
# checking for negative value in feature "median_sale_price"
(df['median_list_price'] < 0).sum().sum()

In [ ]:
#pip install missingno

In [ ]:
test=df[['median_list_price']]
msno.matrix(test)

We can see that data is missing through out the instances ( The NA values are not back to back only at the tail or in the middle of the dataset)

In [ ]:
# Imputing median_list_price with its Mean as we can't perform other technique such as KNN as we are considering to keep the outlier for this feature 
df['median_list_price'].fillna(df['median_list_price'].mean(), inplace = True)

In [ ]:
missing_vals2 = df['median_list_price'].isna().sum()/len(df['median_sale_price'])*100
missing_vals2

In [ ]:

count = 1;
min_value = sys.maxsize
max_value = -1
print(min_value)
for key, value in df['median_list_price'].iteritems():
    if value < min_value:
      min_value = value

    if value > max_value:
      max_value = value
    
print("Minimum value", min_value)
print("Maximum value", max_value)

In [ ]:
#checking for how many value outliers dropped
median_list_price_df=df['median_list_price']
value=1150000.0
#maximum value after removing outliers
count=0
for i, number in df['median_list_price'].iteritems():
    if number > value:
        count=count+1
print(count)
#Percentage of outliers dropped
print("Percentage of outliers dropped",str((count*100)/df['median_list_price'].shape[0])) 

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['median_list_price'].plot(kind="box")

In [ ]:

count = 1;
min_value = sys.maxsize
max_value = -1
print(min_value)
for key, value in df['median_list_price'].iteritems():
    if value < min_value:
      min_value = value

    if value > max_value:
      max_value = value
    
print("Minimum value", min_value)
print("Maximum value", max_value)

In [ ]:
df['median_list_price'].plot(kind='kde')

# median_ppsf feature cleaning

In [ ]:
df['median_ppsf'].value_counts()

In [ ]:
df['median_ppsf'].unique()

In [ ]:
#checking for negative value for this feature
(df['median_ppsf']<0).sum().sum()

In [ ]:
count = 1;
min_value = sys.maxsize
max_value = -1
print(min_value)
for key, value in df['median_ppsf'].iteritems():
    if value < min_value:
      min_value = value

    if value > max_value:
      max_value = value
    
print("Minimum value", min_value)
print("Maximum value", max_value)

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['median_ppsf'].plot(kind="box")

# median_list_ppsf feature cleaning

In [ ]:
df['median_list_ppsf'].value_counts()

In [ ]:
#checking for negative value for this feature
(df['median_list_ppsf']<0).sum().sum()

In [ ]:

test=df[['median_list_ppsf']]
msno.matrix(test)

Same justification for this as mentioned for abive feature

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['median_list_ppsf'].plot(kind="box")

In [ ]:
count = 1;
min_value = sys.maxsize
max_value = -1
print(min_value)
for key, value in df['median_list_ppsf'].iteritems():
    if value < min_value:
      min_value = value

    if value > max_value:
      max_value = value
    
print("Minimum value", min_value)
print("Maximum value", max_value)

this is certainly an outlier because this is about median per square feet for listed house and max value is 5500000.0

In [ ]:
df.shape

In [ ]:
# Fix the index to be in order
df.reset_index(inplace=True)
df.drop('index', axis= 1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

# City Feature cleaning

In [ ]:
df['city'].value_counts()

In [ ]:
len(df['city'].str.lower().unique())

In [ ]:
df['city'].str.lower()

In [ ]:
# performing labelencoding on city column
label_city = LabelEncoder()
df['city']=label_city.fit_transform(df['city'])
# df = df.drop('city',axis = 1)
# df = df.join(one_hot_city)

In [ ]:
print(df['city'].unique())

# State,parent_metro_region and state_code cleaning

In [ ]:
#performing one hot encoding on state column
one_hot_state = pd.get_dummies(df['state'])
one_hot_state_code = pd.get_dummies(df['state_code'])
df = df.drop('state',axis = 1)
df = df.drop('state_code',axis = 1)
df = df.join(one_hot_state_code)
df = df.join(one_hot_state)


In [ ]:
df['parent_metro_region'].unique()


In [ ]:
# split  Parent metro region into metro region and drop CA
df["parent_metro_region"] = df["parent_metro_region"].str.replace(r',[^,]*$', '')

In [ ]:
df['parent_metro_region'].unique()

In [ ]:
one_hot_parent_metro_region = pd.get_dummies(df['parent_metro_region'])
df = df.drop('parent_metro_region' ,axis=1)
df = df.join(one_hot_parent_metro_region)

In [ ]:
df.head(5)

# 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'months_of_supply',  'avg_sale_to_list', 'sold_above_list', 'price_drops', 'off_market_in_two_weeks features cleaning

In [ ]:
df['homes_sold'].value_counts()

In [ ]:
#checking for negative value for this feature # checked for each of these features and got zero in results
col=['homes_sold','pending_sales','new_listings','inventory','months_of_supply','avg_sale_to_list','sold_above_list','price_drops','off_market_in_two_weeks']
(df[col]<0).sum().sum()

In [ ]:
col=['homes_sold','pending_sales','new_listings','inventory','months_of_supply','avg_sale_to_list','sold_above_list','price_drops','off_market_in_two_weeks']
(df[col]<0).sum().sum()
missing_vals2 = df[col].isna().sum()/len(df[col])*100
missing_vals2

In [ ]:
col=['homes_sold','pending_sales','new_listings','inventory','months_of_supply','avg_sale_to_list','sold_above_list','price_drops','off_market_in_two_weeks']
test=df[col]
msno.matrix(test)

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['pending_sales'].plot(kind="box")

In [ ]:
Q1 = df['pending_sales'].quantile(0.25)
Q3 = df['pending_sales'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['pending_sales'] < (Q1 - 1.5 * IQR)) |(df['pending_sales'] > (Q3 + 1.5* IQR)))]

In [ ]:
df['pending_sales'].fillna(df['pending_sales'].mean(), inplace = True)


In [ ]:
df['new_listings'].fillna(df['new_listings'].mean(), inplace = True)


In [ ]:
df['inventory'].fillna(df['inventory'].mean(), inplace = True)

In [ ]:
df['months_of_supply'].fillna(df['months_of_supply'].mean(), inplace = True)

In [ ]:
df['price_drops'].fillna(df['price_drops'].mean(), inplace = True)

In [ ]:
df['off_market_in_two_weeks'].fillna(df['off_market_in_two_weeks'].mean(), inplace = True)

In [ ]:
df['median_dom'].fillna(df['median_dom'].mean(), inplace = True)

In [ ]:
df['median_ppsf'].fillna(df['median_ppsf'].mean(), inplace = True)

In [ ]:
df['median_list_ppsf'].fillna(df['median_list_ppsf'].mean(), inplace = True)

In [ ]:
df['avg_sale_to_list'].fillna(df['avg_sale_to_list'].mean(), inplace = True)

In [ ]:
msno.matrix(df)

In [ ]:
missing_vals2 = df[col].isna().sum()/len(df[col])*100
missing_vals2

In [ ]:
# # Normalize Distance and Age
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# cols_to_norm = ['median_sale_price', 'median_list_price', 'median_ppsf','median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'months_of_supply', 'avg_sale_to_list', 'sold_above_list', 'price_drops', 'off_market_in_two_weeks','MORTGAGE30US']
# df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])

In [ ]:
cols = ['median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'months_of_supply','avg_sale_to_list', 'sold_above_list', 'price_drops'] # one or more

Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# Baseline Model

In [ ]:
df['median_dom'].value_counts()

In [ ]:
#checking for negative value for this feature
(df['median_dom']<0).sum().sum()

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['median_dom'].plot(kind="box")

In [ ]:
Q1 = df['median_dom'].quantile(0.25)
Q3 = df['median_dom'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['median_dom'] < (Q1 - 1.5 * IQR)) |(df['median_dom'] > (Q3 + 1.5* IQR)))]

In [ ]:
#checking for outliers
#box plot before removing outliers
fig, axes = plt.subplots(1, 1, figsize=(5,5))
fig.suptitle('Detecting Feature Value Outliers')
#Visualize the median_list_ppsf
df['median_dom'].plot(kind="box")

In [ ]:
df['median_dom'].shape

In [ ]:
df['median_dom'].fillna(df['median_dom'].mean(), inplace = True)

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.hist(df['median_dom'], bins=100, density=True, alpha=0.5, color='b')
  
plt.show()

In [ ]:
df['median_dom'] = np.round(df['median_dom'], decimals = 1)

In [ ]:
df.columns

In [ ]:
df['median_dom'].describe()

We have used apply() on the entire row of the dataframe and among all, we have taken onyl median_dom

In [ ]:
def assign_quality(row):
    if row['median_dom'] <= 21: #3 weeks...
        return 0
    elif row['median_dom'] > 21 and row['median_dom'] <= 45: #3 weeks to 1.5 months...
        return 1
    else: # greater than 1.5 months...
        return 2

In [ ]:
df['quality'] = df.apply(assign_quality, axis=1)

In [ ]:
df1 = df[['median_dom', 'quality']]

In [ ]:
df1['quality'].unique()

In [ ]:
df1['quality'].value_counts()

In [ ]:
df.reset_index(inplace=True)
df.drop('index', axis= 1, inplace=True)

In [ ]:
# df['quality_Ranking'] = 0
# for i in range(len(df)):
#     if df['quality'][i] == 'HIGH':
#         df['quality_Ranking'][i] = 1
#     elif df['quality'][i] == 'MEDIUM':
#         df['quality_Ranking'][i] = 2
#     else :
#         df['quality_Ranking'][i]=3


In [ ]:
df.head()

In [ ]:
df['quality']

In [ ]:
df['parent_metro_region_metro_code'].unique()

In [ ]:
for columns in df.columns:
    print(column)
    

In [ ]:
#Perform normalization on non one hot encoding columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
col=['median_sale_price', 'median_list_price', 'median_ppsf',
       'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings',
       'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list',
       'sold_above_list', 'price_drops', 'off_market_in_two_weeks']



df[col] = scaler.fit_transform(df[col])

In [ ]:
one_hot_parent_metro_region_metro_code = pd.get_dummies(df['parent_metro_region_metro_code'])
df = df.drop('parent_metro_region_metro_code' ,axis=1)
df = df.join(one_hot_parent_metro_region_metro_code)

In [ ]:
df.shape

In [ ]:
#Save the cleaned data to a File...
#df.to_csv('C:/Users/saumy/Desktop/data.csv',encoding='utf-8-sig')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [ ]:
y = df['quality']
X = df.drop('quality',axis=1)
#, x_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
print(len(y_val))
print(len(X_val))

logistic regression predicts if something is true or false,curve goes from 0 to1 and it tells the probability and is usally used for classification problem

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#print(classification_report(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_val)


In [ ]:
print(classification_report(y_val, y_pred))


# Performing feature reduction using RFE

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
#2. Using Variance Threshold for feature selection
selector = VarianceThreshold(threshold=(.9 * (1 - .9)));
temp = selector.fit_transform(df);

In [ ]:
temp.shape

In [ ]:
selected_features = selector.get_support(indices=True)

In [ ]:
reduceddf = df.iloc[:,selected_features]

In [ ]:
reduceddf.columns

In [ ]:
y1 = reduceddf['quality']
X1 = reduceddf.drop('quality',axis=1)
#, x_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=42)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.1, random_state=1)

X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.1, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
#y_label = reduceddf['quality']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(reduceddf,y_label, test_size=0.40,random_state=15,stratify=y_label)

In [ ]:
model.fit(X1_train,y1_train)
y1_pred = model.predict(X1_test)

In [ ]:
print(classification_report(y_test, y_pred))